In [6]:
# imports
import time
import pandas as pd
import bs4
import requests
import datetime

In [7]:
def get_soup(url: str):
    """ ### get_soup
    Faz uma requisição GET para a URL passada como parâmetro e
    retorna um objeto do tipo BeautifulSoup
    """

    html_content = requests.get(url).text
    soup = bs4.BeautifulSoup(html_content, 'lxml')
    return soup


In [8]:
def load_soap_data(soap_data, proposicao:int, ano:int):
    """ ### load_soap_data
    Interpreta os dados oriundos de uma página HTML
    parâmetros:
    * soap_data: objeto do tipo BeautifulSoup com os dados do HTML
    * proposicao: número da proposição
    * ano: ano da proposição (data inicial: 1997)
    """

    # obtém a lista de chave e valores oriundos do body da pagina
    keys = soap_data.find_all('dt')
    values = soap_data.find_all('dd')
    values = soap_data.find_all('dd')

    # cria um dict baseando-se na zipagem das listas
    dados = {
      key.get_text().lower(): value.get_text()
      for key, value in zip(keys, values)
    }

    # obtem o texto do paragrafo
    texto = soap_data.find('p').get_text()

    # cria um novos keys para armazenar os dados extras
    dados['texto'] = texto
    dados['proposicao'] = proposicao
    dados['ano'] = ano

    return dados

In [9]:
def get_data():
    """### get_data
    Obtém dados a partir do web scraping da pagina *legislador*
    """

    # define mecanismos de manipulação da URL
    url_base = "https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nr"
    url_dynamic =  lambda prop, ano: f'{url_base}Proposicao={prop}&aaProposicao={ano}'

    # define dadas de inicio e fim da execução
    end_year = datetime.datetime.now().year + 1
    start_year = 2022

    result = []

    for ano in range(start_year, end_year):
        proposicao = 1
        raw_data = get_soup( url_dynamic(proposicao, ano) )
        has_data = len(raw_data.findAll('dt')) > 0

        while has_data:

            # para o processo apenas para agilizar os testes
            if proposicao >= 11: break

            result.append(load_soap_data(raw_data, proposicao, ano))

            proposicao += 1
            raw_data = get_soup( url_dynamic(proposicao, ano) )
            has_data = len(raw_data.findAll('dt')) > 0

            print('proposicao:', f'{ano}-{proposicao}', end='\r')

        print(f'Total de Preposições em {ano}:', proposicao)


    return result

In [13]:
preposicoes = get_data()
len(preposicoes)

Total de Preposições em 2022: 11
Total de Preposições em 2023: 11


20

In [12]:
preposicoes[0]

{'reunião': '03/02/2022',
 'deliberação': '03/02/2022',
 'situação': 'Proposição Despachada',
 'assunto': 'Limpeza, Macadamização, Patrolamento, Retificação; Alargamento',
 'autor': 'Vereador Anderson Luz dos Santos.',
 'texto': 'O vereador que esta subscreve, no uso das atribuições que lhe confere o Regimento Interno desta Casa Legislativa, vem requerer, após ouvido o colendo Plenário, encaminhamento de cópia da presente Indicação à Secretaria de Obras sugerindo o exposto a seguir: Manutenção da pista (fechar buraco) na rua Izabel Wolff, bairro Estrada das Areias (imagem anexa).',
 'proposicao': 1,
 'ano': 2022}